# Introduction and Deployment of VLA: SMolVLA

### Lab Table of Contents
1. [1_chatgpt.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/1_chatgpt.ipynb)
2. [2_CLIP.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/2_CLIP.ipynb)
3. [3_VLM_BLIP.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/3_VLM_BLIP.ipynb)
4. **[4_VLA.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/4_VLA.ipynb)**
5. [5_safety.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/5_safety.ipynb)
6. [Lab Checkoff](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/checkoff.txt)

## VLA: Vision-Language Action Model
Compared to VLMs, a VLA is specifically made for robotics where you want to be able to use language to help understand the correct action a robot should take.

SmolVLA is an example of a VLA foundation model where inputs to the model are camera views, the robot's current state, and natural language instruction.

For a more comprehensive overview on all LeRobot has to offer, [LeRobot Repo](https://learnopencv.com/vision-language-action-models-lerobot-policy/). 

#### Ensure you have a new environment Set-Up:
* `conda create -n <env_name> python=3.10`

Walk through each cell of the notebook to better understand the SmolVLA model. After running all cells, you should see an (image, action) pair. Reflect on the following questions.
1. Does the action make sense based on the input image displayed? 
2. How clear or ambiguous is the action token? 
3. What challenges or safety concerns might we face if the output action token is ambiguous?
4. How would you make the action token more clear? 

In [ ]:
# Install Dependencies

!pip install ipykernel matplotlib
!pip install lerobot
!conda install ffmpeg -c conda-forge -y

### Import Libraries

In [ ]:
import torch
import matplotlib.pyplot as plt
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
from lerobot.common.policies.lerobot_policy import LeRobotPolicy

def show_image(img, instruction):
    """Display an image with its instruction."""
    plt.imshow(img.permute(1, 2, 0))  # Convert from (C, H, W) to (H, W, C)
    plt.title(f"Instruction: \"{instruction}\"")
    plt.axis('off')
    plt.show()

### Load the Pre-Trained VLA (SmolVLA)

In [ ]:
device = torch.device("cpu")
print(f"Using device: {device}")

# Load the pre-trained SmolVLA policy
# This downloads the model weights from the Hugging Face Hub
policy = LeRobotPolicy.from_pretrained("lerobot/smolvla_base").to(device)

# Set the model to evaluation mode (disables things like dropout)
policy.eval()

print("SmolVLA model loaded successfully!")

### Load Sample Data

In [ ]:
dataset_repo_id = "lerobot/svla_so100_pickplace"
dataset = LeRobotDataset(dataset_repo_id)

print(f"Loaded dataset with {len(dataset)} total steps.")

# Let's get a single sample from the dataset (e.g., sample #1000)
sample_idx = 1000
sample = dataset[sample_idx]

# Let's see what's in our sample
print("\nSample keys:")
print(sample.keys())

# The 'observation' key contains the inputs for the model
print("\nObservation keys:")
print(sample['observation'].keys())

### Prepare Inputs and Run Inference

In [ ]:
# Get the instruction and observation from our sample
instruction = sample['instruction']

# Prepare the observation dictionary
# We add a batch dimension (B) to each tensor, e.g., (C, H, W) -> (B, C, H, W)
observation = {
    'images.top': sample['observation']['images.top'].to(device).unsqueeze(0),
    'state': sample['observation']['state'].to(device).unsqueeze(0)
}

# Run inference!
# We use torch.no_grad() to tell PyTorch we're not training, which saves memory.
with torch.no_grad():
    action = policy.select_action(observation, instruction)

print(f"Instruction: \"{instruction}\"")
print(f"\nPredicted Action: {action.cpu().numpy()}")

# Let's also see what the "ground truth" action was (what the human did)
ground_truth_action = sample['action'].numpy()
print(f"Ground Truth Action: {ground_truth_action}")

### Visualize the Result

In [ ]:
top_image = observation['images.top'][0].cpu()

# Use our helper function to show the image and instruction
show_image(top_image, instruction)

## References

* [LeRobot VLA Demo](https://learnopencv.com/vision-language-action-models-lerobot-policy/)
* [SmolVLA](https://smolvla.net/index_en)
* [HuggingFace SmolVLA Blog](https://huggingface.co/blog/smolvla)